# Data Visualization Interfaces with Python Dash
![](images/pexels.jpg)

# Setup

In [1]:
import logging
import time
import warnings

import catboost as cb
import datatable as dt
import joblib
import lightgbm as lgbm
import matplotlib.pyplot as plt
import numpy as np
import optuna
import pandas as pd
import seaborn as sns
import shap
import umap
import umap.plot
import xgboost as xgb
from optuna.samplers import TPESampler
from sklearn.compose import *
from sklearn.impute import *
from sklearn.metrics import *
from sklearn.model_selection import *
from sklearn.pipeline import Pipeline, make_pipeline
from sklearn.preprocessing import *

logging.basicConfig(
    format="%(asctime)s - %(message)s", datefmt="%d-%b-%y %H:%M:%S", level=logging.INFO
)
optuna.logging.set_verbosity(optuna.logging.WARNING)
warnings.filterwarnings("ignore")
pd.set_option("float_format", "{:.5f}".format)

# Introduction

Data visualization interfacing, or dashboarding is an important part of data analysts' skillset. Dashboards or data apps are used everywhere now, from reporting your analysis through a series of visuals to showcasing your machine learning apps. When there is data involved, so is Python. Specifically, we are talking about its Dash library, which is built on top one of the hottest graphing libraries - Plotly. 

Dash makes it a breeze to create and share your data analysis through interactive dashboards using only Python code. There is no need to learn HTML, CSS or complex JavaScript frameworks like React.js. In this tutorial, we will get a glimpse of what Dash can do and how you can integrate it into your own workflow.

# Installation

Dash can be installed with a single `pip` install. Here, we install `pandas` library as well to work with out datasets:

```
pip install dash pandas
```

The above command also installs `plotly` data visualization library. Currently, it is one of the hottest graphing libraries in Python ecosystem, mainly for its interactive charts. Plotly and Dash are both created by Plotly Software Foundation, so the libraries work pretty well together. 

# Requirements to using Dash

A powerful framework like Dash has a few requirements from its users to get the most out of its features. First and foremost, you must know Plotly Python as Dash can only display Plotly's interactive charts. 

Next, a basic understanding of HTML and CSS is also a must. Dash is like React.js but only in Python. It is a template framework where you can build a data website without JavaScript. 

You must understand that a dashboard contains multiple visuals and it is up to the user how all these visuals are displayed in a single page. In other words, you decide the layout of your visuals as a user. The visuals itself are handled by Plotly but the layout part, it is all up to Dash. 

We will see examples of these concepts in the coming sections.

# Intro to Dash apps

Let's create out very first Dash app. After the installation, we import the following libraries:

```python
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd
```

`dash` is the global library containing all the core features. The `dash_core_components` and `dash_html_components` are libraries that are installed with dash. They contain Dash-specific features and Python representation of HTML components (tags). More on them later.

Any Dash app starts with the following command:

In [9]:
app = dash.Dash(name="my_first_dash_app")
app

The above code creates all the boilerplate code for a blank website. We don't want a blank page, so let's populate it.

First, we load a built-in dataset from Plotly and create a simple scatterplot:

```python
# Load dataset using Plotly
tips = px.data.tips()

fig = px.scatter(tips, x="total_bill", y="tip") # Create a scatterplot
```

Then, we add this figure to our app's `layout` attribute inside a `div` tag with a few texts:

```python
app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),  # Create a title with H1 tag

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),  # Display some text

    dcc.Graph(
        id='example-graph',
        figure=fig
    )  # Display the Plotly figure
])

if __name__ == '__main__':
    app.run_server(debug=True) # Run the Dash app

```

If you notice, we are creating the HTML tags using the `dash_html_components (html)` library and the figure using the core components library. 

The H1 title tag is followed by a div that contains a simple text, then the graph itself using the `Graph` function of `dcc`. All of these are inside a single DIV tag's `children` attribute. At the end, we are also adding the command which runs our app in debug mode, meaning the changes take effect as the script changes. Here is the complete code so far:

```python
import dash
import dash_core_components as dcc
import dash_html_components as html
import plotly.express as px
import pandas as pd

# Create the app
app = dash.Dash(__name__)

# Load dataset using Plotly
tips = px.data.tips()

fig = px.scatter(tips, x="total_bill", y="tip") # Create a scatterplot

app.layout = html.Div(children=[
    html.H1(children='Hello Dash'),  # Create a title with H1 tag

    html.Div(children='''
        Dash: A web application framework for your data.
    '''),  # Display some text

    dcc.Graph(
        id='example-graph',
        figure=fig
    )  # Display the Plotly figure
])

if __name__ == '__main__':
    app.run_server(debug=True) # Run the Dash app

```

Put it inside a Python script and run it. You will get a message in the terminal telling you to go to this link: http://127.0.0.1:8050/

So, let's go:

![](images/1.png)

In the next sections, we will get down to the hairy details of what we did here and once the dust settles, you will hopefully get out with a solid knowledge of how Dash works and most importantly, how to use it.

# app.layout

Let's start with the `layout` attribute. It is the single attribute that contains all your HTML components and figures. All your visuals and HTML tags should be passed to it in a final DIV tag.

Depending on the size of your project, this attribute can get pretty humongous, so I recommend creating all your HTML tags and figures in separate variables and then, pass them to `layout`. For example, here is what the above app would look like if you follow this tip:

In [12]:
app = dash.Dash(name="app")

# Load dataset using Plotly
tips = px.data.tips()

fig = px.scatter(tips, x="total_bill", y="tip")  # Create a scatterplot

title = html.H1("Hello Dash!")
text_div = html.Div("Dash: A web application framework for your data.")
graph_to_display = dcc.Graph(id="scatter", figure=fig)

app.layout = html.Div(children=[title, text_div, graph_to_display])

Now, this was much neater and more compact, right? This is one of the things that you won't learn from the Dash docs. It is full of nested code instead of doing the above.

# HTML and CSS components in Dash

Now, let's discuss how HTML and CSS works in Dash. As I mentioned, the `dash_html_components` sub-library contains the most common HTML tags such as divs, buttons, text boxes, captions, title tags (h1-6), etc. 

All these are implemented in Python code under relevant names as a representation of their HTML counterparts. So, a code like below:

```python
import dash_html_components as html

html.Div([
    html.H1('Hello Dash'),
    html.Div([
        html.P('Dash converts Python classes into HTML'),
        html.P("This conversion happens behind the scenes by Dash's JavaScript front-end")
    ])
])
```

will be interpreted like this by your browser:

```html
<div>
    <h1>Hello Dash</h1>
    <div>
        <p>Dash converts Python classes into HTML</p>
        <p>This conversion happens behind the scenes by Dash's JavaScript front-end</p>
    </div>
</div>
```

All HTML tags under this sub-library contains these general arguments:

- `id` - the same as `id` attribute of HTML tags
- `className` - the same as `class` attribute of HTML tags
- `style` - the same as `style` attribute of HTML tags but only accepts a dictionary of CSS styles
- `children` - the first argument of most HTML components

Here is an example `div` with a few customizations:

```python
app = dash.Dash(name="app")

app.layout = html.Div(
    children=html.H3("Simple Div"),
    id="sample_div",
    className="red_div",
    style={"backgroundColor": "red"},
)

if __name__ == "__main__":
    app.run_server(debug=True)
```

![](images/2.png)

The `children` argument is special - it can accept accepts and strings but most of the time pass other HTML components inside a list if it is a [container tag](https://www.w3schools.com/w3css/w3css_containers.asp) like a `div`.

> A note on CSS styles: most CSS style attributes use hyphens to break words. When you pass them to Dash's `style` argument, they should follow camelCase like backgroundColor.

I highly recommend you learn about these HTML tags as these are the only things that keep together the layout of your app. Here is a [complete list](https://dash.plotly.com/dash-html-components) of HTML tags Dash supports.

# Dash core components

Another important part of Dash is its core components. The `dash_core_components` library contains a few other HTML tags but already built-in with a few CSS and JavaScript flairs. Some examples include dropdowns, sliders, download and upload functionalities and the one we have already seen, which was a component to display Plotly Graphs.

Here are a few examples of these components, starting with a dropdown:

```python
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        options=[
            {'label': 'FC Barcelona', 'value': 'FCB'},
            {'label': 'Real Madrid', 'value': 'RM'},
            {'label': 'Manchester United', 'value': 'MU'}
        ],
        value='FCB' # The default value to display
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
```

![](images/1.gif)

A multi drowdown:

```python
app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Dropdown(
        options=[
            {'label': 'FC Barcelona', 'value': 'FCB'},
            {'label': 'Real Madrid', 'value': 'RM'},
            {'label': 'Manchester United', 'value': 'MU'}
        ],
        multi=True,
        value="FCB"
    )
], style={"width": 200})

if __name__ == '__main__':
    app.run_server(debug=True)

```

![](images/2.gif)

A slider with labeled breakpoints:

```python
import dash
import dash_core_components as dcc
import dash_html_components as html

app = dash.Dash(__name__)

app.layout = html.Div([
    dcc.Slider(
        min=0,
        max=9,
        marks={i: 'Label{}'.format(i) for i in range(10)},
        value=5,
    )
])

if __name__ == '__main__':
    app.run_server(debug=True)
```

![](images/3.gif)

If you noticed, there is a universal `value` attribute which represents the default value when a component is rendered for the first time.

You can see the full list of core components [here](https://dash.plotly.com/dash-core-components).

# Putting the concepts together

As a final example, take a look at the below app:

```python
import seaborn as sns

app = dash.Dash(__name__)

diamonds = sns.load_dataset("diamonds")

scatter = px.scatter(
    data_frame=diamonds,
    x="price",
    y="carat",
    color="cut",
    title="Carat vs. Price of Diamonds",
    width=600,
    height=400,
)
histogram = px.histogram(
    data_frame=diamonds,
    x="price",
    title="Histogram of Diamond prices",
    width=600,
    height=400,
)
violin = px.violin(
    data_frame=diamonds,
    x="cut",
    y="price",
    title="Violin Plot of Cut vs. Price",
    width=600,
    height=400,
)

left_fig = html.Div(children=dcc.Graph(figure=scatter))
right_fig = html.Div(children=dcc.Graph(figure=histogram))

upper_div = html.Div([left_fig, right_fig], style={"display": "flex"})
central_div = html.Div(
    children=dcc.Graph(figure=violin),
    style={"display": "flex", "justify-content": "center"},
)
app.layout = html.Div([upper_div, central_div])

if __name__ == "__main__":
    app.run_server(debug=True)
    
```

We import the Diamonds dataset from Seaborn and create three charts - a scatterplot, a histogram and a violin plot. We want to display the scatterplot and violinplot next to each other and put the violin plot right below in the center. 

For this, we are creating two divs to contain the scatter plot and violin plot - `left_figure` and `right_figure`. Then, these two divs are put into another div - `upper_div` for simplicity. We are setting a CSS style of flex-box to this div, which puts the figures side-by-side like we wanted. 

Then, we are creating the central div that contains the violin plot and we are centering it using flex-box and its `justify-content` attribute. 

Finally, we put everything into the `layout` and run the script. Here is the final output:

![](images/4.gif)

# Conclusion

Here is a summary of the steps to create a basic Dash app:

1. Create the app with `dash.Dash` and give it any name.
2. Sketch the layout of your plots in the dashboard before writing the actual code.
3. Create the plots that go into your dashboard.
4. Create the boilerplate layout using HTML Dash components.
5. Add your figures to their relevant containers.
6. Finally, add all HTML components to the `layout` attribute.

Even though we have covered a lot of basics like HTML, core components and app layout, we have barely scratched the surface of what Dash can do. 

For example, even though I showed you many examples of interactive HTML components, I didn't show how you can integrate them into your app. For example, how can you update your graphs based on user input like sliders, text inputs or something similar?

That's where callbacks come in - a powerful and central feature of Dash. The basic idea of callbacks is that you can define functions in Dash that gets fired off when a component gets changed by the user and the function modifies another component based on this event. 

Dash devotes a large section in the docs just to explain callbacks as they are hard to wrap your mind around at first. I suggest you go [there](https://dash.plotly.com/#:~:text=Dash%20App%20Gallery-,Dash%20Tutorial,-Installation) as the next step. I also recommend you check out the [example gallery](https://dash.gallery/Portal/) where you can see and enjoy some very cool projects built with Dash. Best part is that many of them are open-source, meaning you can learn much from their source codes. Thanks for reading!